In [17]:
from collections import defaultdict
import pandas as pd
import pickle
import re
from tqdm.auto import tqdm

In [3]:
stim_df = pd.read_csv("adopted_df_blocks_withBalancedUnion.csv")
stim_df

,word_us,word_length,ortho_n_dens_s,ortho_n_freq_s_m,ortho_n_freq_s_sd,ortho_upoint,old20_m,old20_sd,sum_bigram,n_phon,...,snd_arc,aff_val_m,aff_val_sd,aff_arou_m,aff_arou_sd,aff_dom_m,aff_dom_sd,Unnamed: 30,COND,block
0,waist,5,1,2.722634,NaN,6,1.90,0.307794,0.031406,4,...,0.549633,5.32,2.16,4.00,2.38,6.05,1.99,NaN,O-P+,4
1,scene,5,2,1.949637,0.756128,6,1.90,0.307794,0.022991,3,...,0.674344,5.94,1.30,1.95,1.50,5.29,2.02,NaN,O-P+,2
2,honey,5,3,2.797754,1.515747,6,1.80,0.410391,0.043430,4,...,0.590188,7.27,1.66,4.38,2.65,6.39,2.12,NaN,O-P+,1
3,clean,5,3,1.927184,1.800658,6,1.75,0.444262,0.035456,4,...,0.644040,7.09,1.80,3.57,2.20,6.11,2.00,NaN,O-P+,3
4,coax,4,3,2.650531,0.628477,5,1.85,0.366348,0.044137,4,...,0.303013,5.26,2.00,4.48,1.97,5.05,1.78,NaN,O-P+,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,sage,4,13,2.836330,1.015031,5,1.15,0.366348,0.017090,3,...,0.573461,5.79,1.69,3.33,2.03,5.25,2.62,NaN,O+P-,4
128,milk,4,7,2.235695,0.753129,5,1.55,0.510418,0.019776,4,...,0.614093,6.74,1.91,2.33,1.56,6.31,2.33,NaN,O+P-,1
129,price,5,4,3.009041,0.124572,6,1.65,0.489360,0.048255,4,...,0.668441,4.94,1.89,3.40,2.33,4.61,1.69,NaN,O+P-,2
130,task,4,7,2.323764,1.343301,5,1.60,0.502625,0.018843,4,...,0.661362,6.05,1.54,3.83,2.50,6.18,1.71,NaN,O+P-,3


In [42]:
final_word_disjunction = "|".join(stim_df.word_us)
final_word_re = r"\b(" + final_word_disjunction + r")\W*$"
print(final_word_re)

\b(waist|scene|honey|clean|coax|tomb|fluke|doubt|city|frame|cause|motor|feud|bait|earth|board|debt|theme|wrong|crowd|goal|drip|guess|green|stuff|pity|duty|floor|money|sign|phase|force|youth|truth|staff|filth|club|daily|film|quirk|shirt|flesh|shelf|cult|felon|area|evil|chief|girl|storm|style|self|gift|drug|baby|rifle|voice|shift|error|fence|glass|wheel|blood|bench|wood|vice|trail|love|coast|crew|lever|shell|tour|mean|stray|cream|grass|beach|dust|clan|plane|trap|tree|stock|park|deep|boil|spit|wind|suit|door|host|pout|wish|chip|camp|pond|water|hero|joke|moral|clay|poet|dream|cramp|clove|spark|plant|table|blush|harm|bunk|space|gist|bush|wild|east|couch|chin|safe|slave|smoke|real|angle|snow|truck|loft|sage|milk|price|task|grab)\W*$


In [43]:
with open("final_word_re.txt", "w") as f:
    f.write(final_word_re)

In [36]:
!pv corpora/opensubtitles-en.txt | ack "{final_word_re}" --output "\$1\t\$_" - > matches.opensubtitles-en.txt

1	\b(waist|scene|honey|clean|coax|tomb|fluke|doubt|city|frame|cause|motor|feud|bait|earth|board|debt|theme|wrong|crowd|goal|drip|guess|green|stuff|pity|duty|floor|money|sign|phase|force|youth|truth|staff|filth|club|daily|film|quirk|shirt|flesh|shelf|cult|felon|area|evil|chief|girl|storm|style|self|gift|drug|baby|rifle|voice|shift|error|fence|glass|wheel|blood|bench|wood|vice|trail|love|coast|crew|lever|shell|tour|mean|stray|cream|grass|beach|dust|clan|plane|trap|tree|stock|park|deep|boil|spit|wind|suit|door|host|pout|wish|chip|camp|pond|water|hero|joke|moral|clay|poet|dream|cramp|clove|spark|plant|table|blush|harm|bunk|space|gist|bush|wild|east|couch|chin|safe|slave|smoke|real|angle|snow|truck|loft|sage|milk|price|task|grab)\W*$
1	\b(waist|scene|honey|clean|coax|tomb|fluke|doubt|city|frame|cause|motor|feud|bait|earth|board|debt|theme|wrong|crowd|goal|drip|guess|green|stuff|pity|duty|floor|money|sign|phase|force|youth|truth|staff|filth|club|daily|film|quirk|shirt|flesh|shelf|cult|felon|